## 23. Плохая обусловленность метода Кардано


# Постановка задачи: 
1) Реализовать метод Кардано для нахождения корней кубического уравнения $y^3+ay^2+by+c=0$, при этом функция `y1, y2, y3 = cardano(a,b,c)` должна работать как для уравнений с вещественными, так и комплексными коэффициентами.

2) Воспользоваться этим методом для нахождения вещественного корня уравнения
$$
y^3 +3y^2 +\lambda^2y+3\lambda^2 = 0
$$
при различных $\lambda$.

3) Исследовать потерю точности из-за ошибок округления при больших $\lambda$ (в частности, для $\lambda$ порядка величины, обратной машинному эпсилон)

4) Сравните результаты с теми, которые получаются методом Брента `scipy.optimize.brentq` и методом Ньютона `scipy.optimize.newton`.

# Краткое описание метода Кардано:
рассмотрим знаменитые формулы для решения кубического уравнения - *формулы Кардано*,
которые правильнее было бы называть *формулами Ферро-Тартальи-Кардано-Фиора*.

Рассмотрим кубическое уравнение $y^3+ay^2+by+c=0$, где все коэффициенты и неизвестная $y$ в общем случае комплексные.

После замены $y = x-\frac{a}{3}$ в уравнении третьей степени
$y^3+ay^2+by+c=0$ исчезает член с квадратом неизвестной $y$.
Уравнение примет вид
$$
x^3+px+q=0.
$$ 
Его решения можно найти в виде 
$$
x=\alpha + \beta,
$$ 
где $\alpha$, $\beta$ --- некоторые
комплексные числа, 
$$
\alpha=\sqrt[3]{-\frac{q}{2}+\sqrt{\frac{q^2}{4}+\frac{p^3}{27}}},
\qquad
\beta=\sqrt[3]{-\frac{q}{2}-\sqrt{\frac{q^2}{4}+\frac{p^3}{27}}}.
$$
Среди всевозможных комбинаций $\alpha$, $\beta$, получаемых при подстановке всех значений корней кубических,
необходимо выбрать лишь те,
которые удовлетворяют условию $3\alpha\beta+p=0$. 
Легко видеть, что если $\alpha \ne 0$ и $\beta\ne 0$, то
таким образом будет получено $3$ решения: для каждого из трех значений $\alpha$ можно определить единственное $\beta$. 

# Реализация кода
Для начала подключим все необходимые для работы библиотеки

In [34]:
import numpy as np
from math import sin,cos,pi
from scipy import linalg
import matplotlib.pyplot as plt
%matplotlib inline

Запишем реализацию функции вычисления корней кубического уравнения. В парамктрах функции передаются коэффициенты уранения, стоящие перед y^2, y^1 и y^0. С помощью этих коэффициентов мы находим промежуточные результаты p и q, получившиеся в результате подстановки $y = x-\frac{a}{3}$. Далее мы про находим t и w, получившиеся в результате подстановки $ x=t + w $. Для "чистого" вычисления корня кубического(без комплексной части числа) нам необходимо, чтобы авражение под корнем было неотрицательным. Поэтому делаем соответствующие манипуляции (прописываем условия). Если же на вход подавалось комплексное число, данные проверки не требуются. Далее для кажой из переменных t и w находим з корня. Чтобы записать пары мы проверяем каждую из них на "совместимость", то есть подставляем в данное уравнение: $3tw+p=0$. При этом вместо приравнивания к нулю, мы сравниваем реальную и мнимую части. Так как во время вычисления переменных t и w, мы производили такие манипуляции, как деление и извлечение корня, то операции поввлияли на конечный ответ, поэтому мы учитываем это округление таким способом. Так, мы находим 3 пары чисел и подставляем их в уравнение  $ x=t + w $ В моей программе это переменные r0,r1 и r2. И находим корни уравнения, подставляя соответствующие r в y. Также, если во время вычислений мы выяснили, что t или w стали равыми нулю, а это недопусимо при решении данным методом, то учесть данный факт можно специальной проверкой и подстановкой.

In [18]:
def cardano(a,b,c):
    p=(-a**2)/3+b
    q=2/27*a**3-a*b/3+c
    
    if((-q/2+(q**2/4+p**3/27)**.5).imag!=0):
        t=-(-q/2+(q**2/4+p**3/27)**.5)**(1/3)
    else:
        if((-q/2+(q**2/4+p**3/27)**.5)>=0): 
            t=-(-q/2+(q**2/4+p**3/27)**.5)**(1/3)
        else:
            t=(q/2-(q**2/4+p**3/27)**.5)**(1/3)
    
   
    if((-q/2-(q**2/4+p**3/27)**.5).imag!=0):
        w=-(-q/2-(q**2/4+p**3/27)**.5)**(1/3)
    else:
        if((-q/2-(q**2/4+p**3/27)**.5)>=0):
            w=-(-q/2-(q**2/4+p**3/27)**.5)**(1/3)
        else:
            w=(q/2+(q**2/4+p**3/27)**.5)**(1/3)      
    
    kt0=t*complex(0.5,(3**.5)/2)
    kt1=t*complex(-1,0)
    kt2=t*complex(0.5,-(3**.5)/2)
    
    kw0=w*complex(0.5,(3**.5)/2)
    kw1=w*complex(-1,0)
    kw2=w*complex(0.5,-(3**.5)/2)
    
    if(t==0 or w==0):
        r0=kw0+kt0
        r1=kw1+kt1
        r2=kw2+kt2
    else:
        if((3*kt0*kw0+p).real <10**(-14) and (3*kt0*kw0+p).imag <10**(-14)):       
            r0=kt0+kw0
            
        if((3*kt1*kw0+p).real <10**(-14) and (3*kt1*kw0+p).imag <10**(-14)):           
            r0=kt1+kw0
             
        if((3*kt2*kw0+p).real <10**(-14) and (3*kt2*kw0+p).imag <10**(-14)):
            r0=kt2+kw0               
        
        if((3*kt0*kw1+p).real <10**(-14) and (3*kt0*kw1+p).imag <10**(-14)):
            r1=kt0+kw1
     
        if((3*kt1*kw1+p).real <10**(-14) and (3*kt1*kw1+p).imag <10**(-14)):
            r1=kt1+kw1
            
        if((3*kt2*kw1+p).real <10**(-14) and (3*kt2*kw1+p).imag <10**(-14)):
            r1=kt2+kw1
        
        if((3*kt0*kw2+p).real <10**(-14) and (3*kt0*kw2+p).imag <10**(-14)):
            r2=kt0+kw2
              
        if((3*kt1*kw2+p).real <10**(-14) and (3*kt1*kw2+p).imag <10**(-14)):
            r2=kt1+kw2       
            
        if((3*kt2*kw2+p).real <10**(-14) and (3*kt2*kw2+p).imag <10**(-14)):
            r2=kt2+kw2
        
    y0=r0-a/3
    y1=r1-a/3
    y2=r2-a/3    
    return y0,y1,y2

К примеру вычислим данное кубичесекое уравнение с вещесвенными коэффициентами:

In [19]:
y1,y2,y3= cardano(1,2,3)   
print(y1, y2, y3)

(0.13784110182549264+1.527312250886629j) (-1.2756822036509852+0j) (0.13784110182549264-1.527312250886629j)


И уравнение с комплексными коэффициентами:

In [20]:
y1,y2,y3= cardano(1+2j,4+2j,3+1j)   
print(y1, y2, y3)

(-0.04717336206442713-3.1502156046278746j) (-0.19764711489671877+1.291428085402078j) (-0.755179523038854-0.1412124807742029j)


Воспользуемся этим методом для решения уравнения $y^3 +3y^2 +\lambda^2y+3\lambda^2 = 0$ при различных $\lambda$.

In [26]:
l=4
y1,y2,y3= cardano(3,l**2,3*l**2)   
print(y1, y2, y3)

l=6
y1,y2,y3= cardano(3,l**2,3*l**2)   
print(y1, y2, y3)

l=7
y1,y2,y3= cardano(3,l**2,3*l**2)   
print(y1, y2, y3)

l=8
y1,y2,y3= cardano(3,l**2,3*l**2)   
print(y1, y2, y3)


(-2.220446049250313e-16+4j) (-2.9999999999999996+0j) (-2.220446049250313e-16-4j)
6j (-3+0j) -6j
(-2.220446049250313e-16+6.999999999999998j) (-2.9999999999999996+0j) (-2.220446049250313e-16-6.999999999999998j)


UnboundLocalError: local variable 'r0' referenced before assignment

Уже при маленьких значениях нам приходилось учитывать ошибку округления при разных операциях в программе. И чем больше коэффициенты мы вводим, тем большую ошибку получаем. При l=8, то есть при 10^16(порядок величины, больший обратному машинному эпсилон; машинный эпсилон 2^(-52)), программа перестает функционировать.Ошибка округления слишком большая для вычисления данного уравнения. При "ручном" вычислении данного уравнения мы находим 1 вещественный корень =-3.
Сравним результаты с теми, которые получаются методом Брента `scipy.optimize.brentq` и методом Ньютона `scipy.optimize.newton`.

In [54]:
from scipy.optimize import newton, brentq 
def f(x):
    return x**3+3*x**2+l**2*x+3*l**2
def fprime(x):
    return 3*x**2+6*x+l**2

l=3**(8)

a = brentq(f,-100,100)
print(a)
b = newton(f, 3, fprime=fprime)
print(b)


l=0.5*10**(16)

a = brentq(f,-100,100)
print(a)
b = newton(f, 3, fprime=fprime)
print(b)

l=20*10**(16)

a = brentq(f,-100,100)
print(a)
b = newton(f, 3, fprime=fprime)
print(b)

l=100*10**(16)

a = brentq(f,-100,100)
print(a)
b = newton(f, 3, fprime=fprime)
print(b)

l=300*10**(16)

a = brentq(f,-100,100)
print(a)
b = newton(f, 3, fprime=fprime)
print(b)

-3.0
-3.0
-3.0
-3.0
-3.0
-3.0000000000000004
-2.999999999999986
-3.0
-3.0
-3.0


Таким образом, мы можем наблюдать, что отклонения вычисления вещественного корня уравнения методами Брента и Ньютона очень малы.
В то время как метод Кардо даже при небольших значенияx выдает отклонения от "ручного" вычисления корней кубического уравнения, а при коэффициентах больших 10^16 степени, вообще перестает функционировать.